# Split and clean the metadata

### Import data + packages

In [167]:
import pandas as pd

In [190]:
key_genes_pd = pd.read_csv('../data/genes_to_investigate.tsv', sep = '\t')
key_genes = list(key_genes_pd['gene'])

## Unicorn 🦄 dataset

### Import and clean metadata

In [191]:
unicorn_meta1 = pd.read_csv('../data/prince_trial_data/RNAseq/NatureMed_GX_ph2_metadata.csv')
unicorn_meta2 = pd.read_csv('../data/prince_trial_data/RNAseq/PICI0002_ph2_clinical.csv')

In [215]:
def pull_gene(current_rna, unicorn_meta, index, gene):
    unicorn_meta.at[index, str(gene)] = float(current_rna.loc[current_rna['Gene Symbol'] == gene]['TPM'])
    return(unicorn_meta)
    

In [218]:
unicorn_meta = unicorn_meta1.merge(unicorn_meta2, left_on='Deidentified.ID', right_on='Deidentified.ID')
#display(unicorn_meta[:3])
#display(list(unicorn_meta.columns))

unicorn_meta = unicorn_meta[['sample.id', 'gdc.anatomic.site.name', 'Received Nivolumab', 
                             'clinical.observation.os', 'clinical.observation.os.event',
                             'clinical.observation.pfs','clinical.observation.pfs.event', 'Filename']].copy()

#list(unicorn_meta['Cancer Location'])#[:3])

for index, row in unicorn_meta.iterrows():
        
    current_rna = pd.read_csv('../data/prince_trial_data/RNAseq/Data/' + 
                                  str(row['Filename']), sep = '\t')
    
    for gene in key_genes:
        if gene in list(current_rna['Gene Symbol']):
            unicorn_meta = pull_gene(current_rna, unicorn_meta, index, str(gene))

display(unicorn_meta[:3])

/Users/cguccion/miniconda3/envs/basic_jupyter/lib/python3.9/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


,sample.id,gdc.anatomic.site.name,Received Nivolumab,clinical.observation.os,clinical.observation.os.event,clinical.observation.pfs,clinical.observation.pfs.event,Filename,JUNB,CXCL2,...,SH2D1A,TBX21,FOXP3,AGT,ANXA6,LAMB1,CTSG,HSPG2,COL6A6,TNXB
0,A463BM739-001,Pancreas,N,218,True,55,True,RNA_A463BM724-001_tumor_rna_expression_report.tsv,320.5607,53.2051,...,1.3846,2.9145,2.1673,17.5587,181.5466,483.6720,25.4769,385.6111,2.8766,9.5319
1,A463BM452-001,Pancreas,N,966,False,351,False,RNA_A463BM452-001_tumor_rna_expression_report.tsv,317.3665,66.5107,...,1.5510,1.9777,1.9495,36.6928,111.3373,404.8040,11.0742,291.8194,1.8305,10.4458
2,A464BM238-001,Pancreas,N,675,True,168,True,RNA_A464BM232-001_tumor_rna_expression_report.tsv,704.3667,37.9816,...,2.1519,2.8429,2.4881,11.1093,160.4167,361.3943,37.2958,496.0353,1.1737,43.3486


In [219]:
#unicorn_meta.to_csv('Padron_highlightGenes_TPM.tsv', index=False, sep = '\t')
unicorn_meta

,sample.id,gdc.anatomic.site.name,Received Nivolumab,clinical.observation.os,clinical.observation.os.event,clinical.observation.pfs,clinical.observation.pfs.event,Filename,JUNB,CXCL2,...,SH2D1A,TBX21,FOXP3,AGT,ANXA6,LAMB1,CTSG,HSPG2,COL6A6,TNXB
0,A463BM739-001,Pancreas,N,218,True,55,True,RNA_A463BM724-001_tumor_rna_expression_report.tsv,320.5607,53.2051,...,1.3846,2.9145,2.1673,17.5587,181.5466,483.6720,25.4769,385.6111,2.8766,9.5319
1,A463BM452-001,Pancreas,N,966,False,351,False,RNA_A463BM452-001_tumor_rna_expression_report.tsv,317.3665,66.5107,...,1.5510,1.9777,1.9495,36.6928,111.3373,404.8040,11.0742,291.8194,1.8305,10.4458
2,A464BM238-001,Pancreas,N,675,True,168,True,RNA_A464BM232-001_tumor_rna_expression_report.tsv,704.3667,37.9816,...,2.1519,2.8429,2.4881,11.1093,160.4167,361.3943,37.2958,496.0353,1.1737,43.3486
3,K00882FP02_SL10,Liver,Y,834,False,365,True,RNA_A284BR911-001_tumor_rna_expression_report.tsv,233.4267,68.9994,...,0.9498,2.6853,5.5421,521.7334,107.2796,137.4385,2.9898,319.4094,0.7850,9.3330
4,BK00286BL01-8,Liver,Y,620,True,157,True,RNA_BK00286BL01_tumor_rna_expression_report.tsv,314.7002,89.7524,...,0.1945,0.5711,4.8503,471.2405,62.2133,90.7268,1.1775,100.7998,12.5388,4.6829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,BK00374BL01-10,Liver,N,645,False,333,True,RNA_BK00374BL01_tumor_rna_expression_report.tsv,356.6914,175.0957,...,2.0435,5.0942,3.9973,558.7450,85.0309,110.9524,1.8750,270.5588,0.7307,20.4083
59,BK02586TS09,Pancreas,N,441,True,352,True,RNA_BK02586TS_tumor_rna_expression_report.tsv,491.0494,41.0737,...,0.7008,3.8016,0.5316,64.2886,76.0001,232.2361,4.4728,512.2293,1.3429,11.2293
60,BK02587TS10,Lung,Y,375,True,168,True,RNA_BK02587TS_tumor_rna_expression_report.tsv,930.7549,293.4225,...,3.7302,10.3619,4.4689,2.5502,110.6763,247.1344,4.7335,336.7379,54.8428,52.9942
61,BK02592BL01-4,Lymph Node,Y,213,False,219,True,RNA_BK02592BL01_tumor_rna_expression_report.tsv,486.8859,72.5719,...,0.0625,0.2445,0.4836,9.0031,117.8499,436.1682,17.6922,694.1143,0.2073,11.0335


## RNA Seq Urothelial Cancer Dataset
- **Contribution of systemic and somatic factors to clinical response and resistance to PD-L1 blockade in urothelial cancer: An exploratory multi-omic analysis**
- Alexandra Snyder et al PLOS Medicine 2017
- https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1002309
- Github: https://github.com/hammerlab/multi-omic-urothelial-anti-pdl1

In [82]:
bladder_meta1 = pd.read_csv('../data/snyder_PLOSmed_2017/clinical_updated.csv', dtype={'ID': str})
bladder_meta2 = pd.read_csv('../data/snyder_PLOSmed_2017/data_clinical.csv', dtype={'patient_id': str})
bladder_meta3 = pd.read_csv('../data/snyder_PLOSmed_2017/data_kallisto.csv', dtype={'patient_id': str})

In [95]:
bladder_meta = bladder_meta1.merge(bladder_meta2, left_on='ID', right_on='patient_id')

#Correct the file name for kallisto files
bladder_meta['bam_id_rna_tumor'] = bladder_meta['bam_id_rna_tumor'] + '-kallisto'
#display(list(bladder_meta.columns))

### Pull RNA-Seq data

In [93]:
for fn in list(bladder_meta['bam_id_rna_tumor']):
    current_rna = pd.read_csv('../data/snyder_PLOSmed_2017/kallisto/' +
                              str(fn) +  '/abundance.tsv', sep = '\t')
    current_meta = bladder_meta.loc[bladder_meta['bam_id_rna_tumor'] == fn]
    current_rna2 = bladder_meta3.loc[bladder_meta3['patient_id'] == str(list(current_meta['ID'])[0])]
    
    display(current_meta)

    display(current_rna)
    
    display(current_rna2)
    
    break

,ID,Age_x,Sex_x,PD-L1_x,Sequence Path,Timing of Sample (days prior to C1D1),Site of Primary Tumor_x,Smoking_x,Pack Years_x,Time from Diagnosis with Metastatic Disease (months)_x,...,is_progressed,is_progressed_or_deceased,os,patient_id,pfs,progressed,progressed_or_deceased,sample_id_dna_normal,sample_id_dna_tumor,sample_id_rna_tumor
0,0040,71,M,IC0,TURBT,18.0,B,Y,15,22.0,...,True,True,24,0040,20,True,True,0040_S14-38354,M1502385,M1502431


,target_id,length,eff_length,est_counts,tpm
0,ENST00000415118,8,4.25000,0.0,0.000000
1,ENST00000434970,9,5.25000,0.0,0.000000
2,ENST00000448914,13,6.57143,0.0,0.000000
3,ENST00000604642,23,11.28570,0.0,0.000000
4,ENST00000603326,19,9.27273,0.0,0.000000
...,...,...,...,...,...
180248,ENST00000450690,55,17.48050,0.0,0.000000
180249,ENST00000605523,50,18.75930,0.5,0.769978
180250,ENST00000605654,60,16.07890,0.0,0.000000
180251,ENST00000603923,55,17.48050,0.0,0.000000


,patient_id,gene_name,est_counts
0,0040,A1BG,2938.41870
1,0040,A1CF,183.01242
2,0040,A2M,6677.81703
3,0040,A2ML1,240.24860
4,0040,A2MP1,1.00000
...,...,...,...
35433,0040,hsa-mir-150,5.00000
35434,0040,hsa-mir-6723,18.95910
35435,0040,hsa-mir-7162,53.00000
35436,0040,hsa-mir-8078,14.71150
